# Scikit-learn MNIST Model Deployment

 * Wrap a Scikit-learn MNIST python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

## Train locally
 

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, metrics
from sklearn.utils import shuffle
from sklearn.datasets import fetch_mldata
from sklearn.externals import joblib

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

mnist_images = mnist.train.images
mnist_labels = mnist.train.labels
 # To apply a classifier on this data, we need to flatten the image, to
    # turn the data in a (samples, feature) matrix:
n_samples = len(mnist_images)
data = mnist_images.reshape((n_samples, -1))
targets = mnist_labels

data,targets = shuffle(data,targets)
classifier = RandomForestClassifier(n_estimators=30)

# We learn the digits on the first half of the digits
classifier.fit(data[:n_samples // 2], targets[:n_samples // 2])

# Now predict the value of the digit on the second half:
expected = targets[n_samples // 2:]
test_data = data[n_samples // 2:]

print(classifier.score(test_data, expected))

predicted = classifier.predict(data[n_samples // 2:])

print("Classification report for classifier %s:\n%s\n"
          % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

joblib.dump(classifier, 'sk.pkl')



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0.9536
Classification report for classifier RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=

['sk.pkl']

Wrap model using s2i

In [2]:
!s2i build . seldonio/seldon-core-s2i-python3:0.7 sk-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Build completed successfully


In [3]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 sk-mnist:0.1

4ac7945aa20408dcfdabcb47ad081cf06e6007acbe414471d02d3ea06c4d6f36


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[0.75  0.424 0.813 0.267 0.448 0.786 0.436 0.941 0.439 0.716 0.833 0.229
  0.387 0.494 0.99  0.893 0.856 0.865 0.623 0.983 0.238 0.368 0.687 0.722
  0.193 0.509 0.779 0.625 0.362 0.004 0.26  0.58  0.17  0.179 0.903 0.07
  0.754 0.267 0.471 0.9   0.332 0.352 0.775 0.376 0.506 0.993 0.023 0.709
  0.569 0.643 0.075 0.031 0.616 0.152 0.403 0.948 0.366 0.353 0.457 0.371
  0.3   0.596 0.314 0.748 0.724 0.785 0.458 0.404 0.087 0.233 0.468 0.97
  0.938 0.698 0.331 0.669 0.436 0.551 0.941 0.161 0.277 0.149 0.284 0.951
  0.983 0.526 0.893 0.585 0.487 0.58  0.418 0.781 0.6   0.487 0.997 0.093
  0.368 0.841 0.659 0.72  0.243 0.349 0.05  0.796 0.798 0.618 0.863 0.222
  0.446 0.901 0.911 0.789 0.543 0.807 0.363 0.085 0.343 0.006 0.34  0.135
  0.518 0.887 0.706 0.285 0.554 0.426 0.336 0.503 0.089 0.475 0.515 0.552
  0.277 0.251 0.668 0.057 0.251 0.353 0.887 0.466 0.54  0.095 0.685 0.385
  0.353 0.603 0.716 0.984 0.855 0.511 0.308 0.028 0

In [5]:
!docker rm mnist_predictor --force

mnist_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [8]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


In [9]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [10]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [11]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [12]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Mar 13 17:03:15 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        1        1           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-spartakus-volunteer-5554c4d8b6-h8r5x  0/1    ContainerCreating  0         0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     4s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s


NOTES:
NOTES: TODO


In [13]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.7 sk-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Build completed successfully


In [14]:
!kubectl create -f sk_mnist.json

seldondeployment.machinelearning.seldon.io/sk-mnist created


In [15]:
!kubectl rollout status deploy/sk-mnist-single-model-3812de6

deployment "sk-mnist-single-model-3812de6" successfully rolled out


In [16]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

----------------------------------------
SENDING NEW REQUEST:

[[0.846 0.642 0.705 0.841 0.875 0.031 0.775 0.272 0.937 0.267 0.512 0.081
  0.631 0.739 0.378 0.492 0.541 0.229 0.479 0.54  0.465 0.322 0.144 0.199
  0.957 0.025 0.039 0.235 0.563 0.601 0.13  0.148 0.365 0.872 0.574 0.836
  0.252 0.742 0.941 0.202 0.825 0.416 0.822 0.56  0.934 0.367 0.537 0.278
  0.18  0.803 0.399 0.658 0.113 0.33  0.445 0.742 0.434 0.599 0.376 0.797
  0.111 0.245 0.812 0.859 0.609 0.02  0.045 0.514 0.115 0.402 0.616 0.246
  0.241 0.403 0.051 0.11  0.117 0.759 0.506 0.597 0.347 0.704 0.658 0.276
  0.919 0.302 0.83  0.673 0.662 0.313 0.859 0.561 0.936 0.667 0.712 0.453
  0.967 0.113 0.002 0.265 0.446 0.924 0.62  0.215 0.94  0.631 0.003 0.615
  0.104 0.534 0.798 0.36  0.152 0.275 0.513 0.448 0.718 0.928 0.842 0.992
  0.477 0.463 0.894 0.3   0.844 0.488 0.291 0.311 0.453 0.819 0.178 0.959
  0.982 0.05  0.268 0.296 0.152 0.671 0.88  0.818 0.154 0.194 0.212 0.773
  0.907 0.445 0.071 0.83  0.622 0.792 0.659 0.426

In [17]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
